In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim

torch.manual_seed(1)

In [6]:
# For tri-grams context len gth is 2, 3rd word must relate to first two words
CONTEXT_SIZE = 2

# 10 dimensional embeddings, embedding vector has size [n,10], n = number of words in vocabulary
EMBEDDING_DIM = 10

# Para from American Gods
test_sentence = """That is the tale; the rest is detail.There are stories that are true, in which each individual’s 
tale is unique and tragic, and the worst of the tragedy is that we have heard it before, and we cannot allow ourselves 
to feel it too deeply. We build a shell around it like an oyster dealing with a painful particle of grit, coating it with 
smooth pearl layers in order to cope. This is how we walk and talk and function, day in, day out, immune to others’ 
pain and loss. If it were to touch us it would cripple us or make saints of us; but for the most part, it does not touch us. 
We cannot allow it to. Tonight, as you eat, reflect if you can: there are children starving in the world, starving in 
numbers larger than the mind can easily hold, up in the big numbers where an error of a million here, a million there, 
can be forgiven. It may be uncomfortable for you to reflect upon this or it may not, but still, you will eat. There are 
accounts which, if we open our hearts to them, will cut us too deeply. Look — here is a good man, good by his own 
lights and the lights of his friends: he is faithful and true to his wife, he adores and lavishes attention on his little 
children, he cares about his country, he does his job punctiliously, as best he can. So, efficiently and good-naturedly, 
he exterminates Jews: he appreciates the music that plays in the background to pacify them; he advises the Jews not to forget 
their identification numbers as they go into the showers — many people, he tells them, forget their numbers, and take the 
wrong clothes, when they come out of the showers. This calms the Jews: there will be life, they assure themselves, after the 
showers. And they are wrong. Our man supervises the detail taking the bodies to the ovens; and if there is anything he 
feels bad about, it is that he still allows the gassing of vermin to affect him. Were he a truly good man, he knows, he 
would feel nothing but joy, as the earth is cleansed of its pests.
Leave him; he cuts too deep. He is too close to us and it hurts.
Women and men, the old and the young of them: there are so many of them, and so many of their stories are tragedies with 
griefs too deep to be contained, but holding here and there tiny joys, snatched from the darkness like flowers picked by the 
fallen traveler from the side of a cliff.
No man, proclaimed Donne, is an Island, and he was wrong. If we were not islands, we would be lost, drowned in each other’s 
tragedies. We are insulated (a word that means, literally, remember, made into an island) from the tragedy of others, by our 
island nature, and by the repetitive shape and form of the stories. We know the shape, and the shape does not change. 
There was a human being who was born, lived, and then, by some means or other, died. there. You may fill in the details from 
your own experience. As unoriginal as any other tale, as unique as any other life. Lives are snowflakes — unique in detail, 
forming patterns we have seen before, but as like one another as peas in a pod (and have you ever looked at peas in a pod? 
I mean, really looked at them? There’s not a chance you’d mistake one for another, after a minute’s close inspection.)
We need individual stories. Without individuals we see only numbers: a thousand dead, a hundred thousand dead, ‘casualties 
may rise to a million’. With individual stories, the statistics become people — but even that is a lie, for the people 
continue to suffer in numbers that themselves are numbing and meaningless. Look, see the child’s swollen, swollen belly, 
and the flies that crawl at the corners of his eyes, his skeletal limbs: will it make it easier for you to know his name, 
his age, his dreams, his fears? To see him from the inside? And if it does, are we not doing a disservice to his sister, 
who lies in the searing dust beside him, a distorted, distended caricature of a human child. And there, if we feel for them, 
are they now more important to us than a thousand other children touched by the same famine, a thousand other young lives 
who will soon be food for the flies’ own myriad squirming children?
We draw our lines around these moments of pain, and remain upon our islands, and they cannot hurt us. They are covered with 
a smooth, safe, nacreous layer to let them slip, pearl-like, from our souls without real pain. Fiction allows us to slide into 
these other heads, these other places, and look out through other eyes. And then in the tale we stop before we die, or we 
die vicariously and unharmed, and in the world beyond the tale we turn the page or close the book, and we resume our lives.
A life, which is, like any other, unlike any other.""".split()

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


In [11]:
print("Vocabulary length ::: ", len(vocab))

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print("Generated trigrams ::: ", trigrams[:3])


Vocabulary length :::  438
Generated trigrams :::  [(['That', 'is'], 'the'), (['is', 'the'], 'tale;'), (['the', 'tale;'], 'the')]


In [9]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = functional.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(100):
    total_loss = 0
    for context, target in trigrams:

        # Converting each owrd in 1*10 sized tensor tensor
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Run the forward pass, getting log probabilities over next words
        log_probs = model(context_idxs)

        # Compute your loss function. (Again, Torch wants the target word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[5291.958989620209, 5225.31563949585, 5160.828944206238, 5098.625154733658, 5039.238744497299, 4983.467415809631, 4931.942175149918, 4884.7227511405945, 4841.274475693703, 4800.869780898094, 4762.980112910271, 4727.252022862434, 4693.421999454498, 4661.301763653755, 4630.659889936447, 4601.3033452034, 4573.040971755981, 4545.709414839745, 4519.113343119621, 4493.106626152992, 4467.554193854332, 4442.33916926384, 4417.348442077637, 4392.509579539299, 4367.759881019592, 4343.048964858055, 4318.318013429642, 4293.5517427921295, 4268.724505782127, 4243.820360898972, 4218.802584767342, 4193.656110405922, 4168.3736045360565, 4142.905546545982, 4117.238303303719, 4091.349445581436, 4065.2250306606293, 4038.8631279468536, 4012.2589473724365, 3985.359647631645, 3958.1994038820267, 3930.737146615982, 3902.9481624364853, 3874.8254877328873, 3846.367839694023, 3817.579847216606, 3788.4297901391983, 3758.920683860779, 3729.034088075161, 3698.764029622078, 3668.1028084158897, 3637.0456870794296, 360